<a href="https://colab.research.google.com/github/bomb1000/mask_map_analysis/blob/main/%E5%8F%A3%E7%BD%A9%E5%9C%B0%E5%9C%96%E5%88%86%E6%9E%90.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
import requests
import json
import pandas as pd


# 利用 requests 對 API 來源發送一個請求
url = 'https://raw.githubusercontent.com/kiang/pharmacies/master/json/points.json'
response = requests.get(url)

# 將請求回應的內容存成一個字串格式
d = response.text

# 將長得像 json 格式的字串解析成字典或列表
data = json.loads(d)

# print(data)
df = pd.DataFrame(data)

# Display the first 5 rows of the DataFrame
# print(df.head())
print(df)

                   type                                           features
0     FeatureCollection  {'type': 'Feature', 'properties': {'id': '5901...
1     FeatureCollection  {'type': 'Feature', 'properties': {'id': '5901...
2     FeatureCollection  {'type': 'Feature', 'properties': {'id': '5901...
3     FeatureCollection  {'type': 'Feature', 'properties': {'id': '5901...
4     FeatureCollection  {'type': 'Feature', 'properties': {'id': '5901...
...                 ...                                                ...
3288  FeatureCollection  {'type': 'Feature', 'properties': {'id': '5903...
3289  FeatureCollection  {'type': 'Feature', 'properties': {'id': '0291...
3290  FeatureCollection  {'type': 'Feature', 'properties': {'id': '5934...
3291  FeatureCollection  {'type': 'Feature', 'properties': {'id': '5937...
3292  FeatureCollection  {'type': 'Feature', 'properties': {'id': '0145...

[3293 rows x 2 columns]


In [11]:
# 計算各地區的藥局數量

med_count = {}

# 填入欄位名稱
for d in data['features']:
    county = d['properties']['address'][:3]
    if county not in med_count:
         med_count[county] = 0
    for key, value in med_count.items():
      if county == key:
           med_count[county] += 1

print(med_count)
# {'台北市': 123, '新北市': 456 ...}


{'臺北市': 340, '高雄市': 424, '臺中市': 436, '臺南市': 280, '基隆市': 57, '新竹市': 39, '嘉義市': 67, '新北市': 507, '桃園市': 271, '新竹縣': 48, '宜蘭縣': 76, '苗栗縣': 57, '彰化縣': 181, '南投縣': 67, '雲林縣': 131, '嘉義縣': 84, '屏東縣': 140, '澎湖縣': 11, '花蓮縣': 47, '臺東縣': 23, '金門縣': 6, '連江縣': 1}


In [16]:
# 計算每個地區的成人剩餘口罩數量，並且將結果從大到小排列

mask_count = {}

# 填入欄位名稱
for d in data['features']:
    county = d['properties']['address'][:3]
    if county not in mask_count:
         mask_count[county] = 0
    for key, value in mask_count.items():
      if county == key:
           mask_count[county] += d['properties']['mask_adult']

# 將結果從大到小排列
mask_count = dict(sorted(mask_count.items(), key=lambda item: item[1], reverse = True))

print(mask_count)
# {'台北市': 12345, '新北市': 45678 ...}

{'新北市': 677660, '臺中市': 556440, '高雄市': 428940, '臺南市': 408400, '桃園市': 345600, '臺北市': 323660, '彰化縣': 219400, '雲林縣': 199470, '屏東縣': 184250, '苗栗縣': 134180, '嘉義縣': 113180, '嘉義市': 107840, '宜蘭縣': 106900, '南投縣': 91240, '基隆市': 80100, '花蓮縣': 57330, '新竹縣': 53760, '新竹市': 45630, '臺東縣': 28510, '金門縣': 15510, '澎湖縣': 12600, '連江縣': 7930}


# **將資料存入資料庫**

In [21]:
import sqlite3
import datetime

conn = sqlite3.connect('mask.db')
c = conn.cursor()

# 新增且清空資料表
c.execute('''CREATE TABLE IF NOT EXISTS pharmacies
             (city text, counts text, createdAt datetime)''')
c.execute('''DELETE FROM pharmacies''')
conn.commit()

# 新增資料
for d in data['features']:
  city = d['properties']['address'][:3]
  counts = d['properties']['mask_adult'] + d['properties']['mask_child']
  t = datetime.datetime.now()
  print(f"INSERT INTO pharmacies VALUES ('{city}', {counts}, '{t}')")
  c.execute(f"INSERT INTO pharmacies VALUES ('{city}', {counts}, '{t}')")
  conn.commit()

# 查詢資料
c.execute("SELECT * FROM pharmacies")
print(c.fetchall())

conn.commit()
conn.close()

INSERT INTO pharmacies VALUES ('臺北市', 2490, '2023-05-24 12:51:18.594510')
INSERT INTO pharmacies VALUES ('臺北市', 2400, '2023-05-24 12:51:18.604875')
INSERT INTO pharmacies VALUES ('臺北市', 1090, '2023-05-24 12:51:18.613729')
INSERT INTO pharmacies VALUES ('臺北市', 1030, '2023-05-24 12:51:18.622611')
INSERT INTO pharmacies VALUES ('臺北市', 1730, '2023-05-24 12:51:18.630882')
INSERT INTO pharmacies VALUES ('臺北市', 770, '2023-05-24 12:51:18.639572')
INSERT INTO pharmacies VALUES ('臺北市', 2670, '2023-05-24 12:51:18.647934')
INSERT INTO pharmacies VALUES ('臺北市', 590, '2023-05-24 12:51:18.656022')
INSERT INTO pharmacies VALUES ('臺北市', 510, '2023-05-24 12:51:18.664253')
INSERT INTO pharmacies VALUES ('臺北市', 3540, '2023-05-24 12:51:18.673923')
INSERT INTO pharmacies VALUES ('臺北市', 2380, '2023-05-24 12:51:18.682613')
INSERT INTO pharmacies VALUES ('臺北市', 790, '2023-05-24 12:51:18.690773')
INSERT INTO pharmacies VALUES ('臺北市', 1140, '2023-05-24 12:51:18.700608')
INSERT INTO pharmacies VALUES ('臺北市', 1290